<a href="https://colab.research.google.com/github/maheshmokka/2320040135/blob/main/Store_Sales_and_Profit_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os, warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from datetime import datetime


try:
    import statsmodels.api as sm
    from statsmodels.tsa.seasonal import seasonal_decompose
    from statsmodels.tsa.arima.model import ARIMA
    HAS_SM = True
except:
    HAS_SM = False


In [ ]:
!ls "/content/drive/MyDrive"


'Colab Notebooks'  'Sample - Superstore.csv'


In [ ]:
df = pd.read_csv(DATA_PATH, encoding="ISO-8859-1")


In [ ]:

print("Shape:", df.shape)


print("\nColumns:", df.columns.tolist())


print("\nSample Data:")
print(df.head())

print("\nMissing values:")
print(df.isnull().sum())


Shape: (9994, 21)

Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']

Sample Data:
   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
1       2  CA-2016-152156   11/8/2016  11/11/2016    Second Class    CG-12520   
2       3  CA-2016-138688   6/12/2016   6/16/2016    Second Class    DV-13045   
3       4  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   
4       5  US-2015-108966  10/11/2015  10/18/2015  Standard Class    SO-20335   

     Customer Name    Segment        Country             City  ...  \
0      Claire Gute   Consumer  United States        Henderson  ...   
1      Claire Gute   Consumer  United States        Henderson  ...   
2

In [ ]:


data = df.copy()

data.columns = [c.strip() for c in data.columns]






col_alias = {
    "OrderDate": "Order Date",
    "Order_Date": "Order Date",
    "OrderID": "Order ID",
    "Order_Id": "Order ID",
    "CustomerID": "Customer ID",
    "Customer_Id": "Customer ID",
    "SubCategory": "Sub-Category",
    "ProductName": "Product Name",
    "ProductID": "Product ID",
    "UnitPrice": "Unit Price"
}
for k,v in col_alias.items():
    if k in data.columns and v not in data.columns:
        data.rename(columns={k:v}, inplace=True)

In [ ]:
# 4) Type fixes & derived fields

if DATE_COL not in data.columns:
    raise SystemExit(f"Expected date column '{DATE_COL}' not found. Please rename in script.")
data[DATE_COL] = pd.to_datetime(data[DATE_COL], errors="coerce")
data = data.dropna(subset=[DATE_COL])


for numcol in ["Quantity", "Sales", "Discount", "Profit", "Cost", "Unit Price"]:
    if numcol in data.columns:
        data[numcol] = pd.to_numeric(data[numcol], errors="coerce")

if "Quantity" not in data.columns:
    data["Quantity"] = 1
if "Sales" not in data.columns:
    raise SystemExit("Need 'Sales' column in the dataset.")
if "Profit" not in data.columns:

    data["Profit"] = data["Sales"] * (1 - data.get("Discount", pd.Series(0,index=data.index))) * 0.2

if "Unit Price" not in data.columns:
    data["Unit Price"] = data["Sales"] / data["Quantity"].replace(0, np.nan)

data["Profit Margin %"] = np.where(
    data["Sales"] > 0, (data["Profit"] / data["Sales"]) * 100, np.nan
)

# Date parts
data["Year"] = data[DATE_COL].dt.year
data["Month"] = data[DATE_COL].dt.month
data["Month Name"] = data[DATE_COL].dt.strftime("%b")
data["Quarter"] = data[DATE_COL].dt.to_period("Q").astype(str)
data["Week"] = data[DATE_COL].dt.isocalendar().week
data["Weekday"] = data[DATE_COL].dt.day_name()

In [ ]:
# 5) Global KPIs
# -------------------------
orders = data[ID_COL].nunique() if ID_COL in data.columns else len(data)
customers = data[CUST_COL].nunique() if CUST_COL in data.columns else None
total_sales = data["Sales"].sum()
total_profit = data["Profit"].sum()
avg_order_value = data.groupby(ID_COL)["Sales"].sum().mean() if ID_COL in data.columns else np.nan
items_per_order = (data.groupby(ID_COL)["Quantity"].sum().mean()
                   if ID_COL in data.columns else np.nan)
overall_margin = (total_profit / total_sales * 100) if total_sales else np.nan

summary = {
    "Orders": orders,
    "Customers": customers,
    "Total Sales": round(total_sales, 2),
    "Total Profit": round(total_profit, 2),
    "Overall Margin %": round(overall_margin, 2),
    "Avg Order Value": round(avg_order_value, 2) if not np.isnan(avg_order_value) else None,
    "Items/Order": round(items_per_order, 2) if not np.isnan(items_per_order) else None
}
pd.Series(summary).to_csv(OUTPUT_DIR / "kpi_summary.csv")

print("== KPI SUMMARY ==")
for k,v in summary.items():
    print(f"{k}: {v}")

== KPI SUMMARY ==
Orders: 5009
Customers: 793
Total Sales: 2297200.86
Total Profit: 286397.02
Overall Margin %: 12.47
Avg Order Value: 458.61
Items/Order: 7.56


In [ ]:
# 6) Aggregations & Exports
# -------------------------
def topn(group_cols, metric="Sales", n=10):
    g = data.groupby(group_cols, dropna=False, as_index=False)[["Sales","Profit"]].sum()
    g["Profit Margin %"] = np.where(g["Sales"]>0, g["Profit"]/g["Sales"]*100, np.nan)
    g = g.sort_values(metric, ascending=False).head(n)
    return g

if "Product Name" in data.columns:
    top_products = topn(["Product Name"], "Sales", 20)
    top_products.to_csv(OUTPUT_DIR / "top_products.csv", index=False)

if "Category" in data.columns:
    top_categories = topn(["Category"], "Sales", 10)
    top_categories.to_csv(OUTPUT_DIR / "top_categories.csv", index=False)

if "Region" in data.columns:
    top_regions = topn(["Region"], "Sales", 10)
    top_regions.to_csv(OUTPUT_DIR / "top_regions.csv", index=False)

if "Product Name" in data.columns:
    loss_skus = (data.groupby("Product Name", as_index=False)[["Sales","Profit"]]
                    .sum().query("Profit < 0").sort_values("Profit"))
    loss_skus.to_csv(OUTPUT_DIR / "loss_making_products.csv", index=False)


if "Discount" in data.columns:
    corr = data[["Discount","Profit Margin %"]].dropna().corr().iloc[0,1]
    with open(OUTPUT_DIR / "discount_margin_correlation.txt", "w") as f:
        f.write(f"Correlation(Discount, Profit Margin %): {corr:.4f}")

In [ ]:
# 7) Time series & Seasonality
# -------------------------
ts = data.set_index(DATE_COL).sort_index()
sales_m = ts["Sales"].resample("MS").sum()

sales_m.to_csv(OUTPUT_DIR / "monthly_sales.csv")
plt.figure(figsize=(10,5))
sales_m.plot()
plt.title("Monthly Sales")
plt.xlabel("Month")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "monthly_sales.png")
plt.close()

if HAS_SM and len(sales_m.dropna()) >= 24:
    result = seasonal_decompose(sales_m, model="additive", period=12)
    # Trend
    plt.figure(figsize=(10,4))
    result.trend.plot()
    plt.title("Sales Trend")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "sales_trend.png")
    plt.close()
    # Seasonality
    plt.figure(figsize=(10,4))
    result.seasonal.plot()
    plt.title("Seasonality")
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / "sales_seasonality.png")
    plt.close()

if HAS_SM and len(sales_m.dropna()) >= 18:
    train = sales_m.dropna()
    try:
        model = ARIMA(train, order=(1,1,1)).fit()
        forecast = model.forecast(steps=3)
        fc_df = pd.DataFrame({"Forecast": forecast})
        fc_df.to_csv(OUTPUT_DIR / "monthly_sales_forecast.csv")
    except Exception as e:
        with open(OUTPUT_DIR / "forecast_error.txt", "w") as f:
            f.write(str(e))

In [ ]:
# 8) Cohorts & RFM (Customer Value)
# -------------------------
if CUST_COL in data.columns:
    # RFM
    ref_date = data[DATE_COL].max() + pd.Timedelta(days=1)
    rfm = (data
           .groupby(CUST_COL)
           .agg({
               DATE_COL: lambda s: (ref_date - s.max()).days, # Recency
               ID_COL: "nunique" if ID_COL in data.columns else "count",   # Frequency
               "Sales": "sum"    # Monetary
           }).rename(columns={DATE_COL:"RecencyDays", ID_COL:"Frequency", "Sales":"Monetary"}))
    # Score each 1(low) - 5(high)
    rfm["R_Score"] = pd.qcut(rfm["RecencyDays"].rank(method="first"), 5, labels=[5,4,3,2,1]).astype(int)
    rfm["F_Score"] = pd.qcut(rfm["Frequency"].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)
    rfm["M_Score"] = pd.qcut(rfm["Monetary"].rank(method="first"), 5, labels=[1,2,3,4,5]).astype(int)
    rfm["RFM_Segment"] = rfm["R_Score"].astype(str)+rfm["F_Score"].astype(str)+rfm["M_Score"].astype(str)
    rfm["RFM_Score"] = rfm[["R_Score","F_Score","M_Score"]].sum(axis=1)
    rfm.to_csv(OUTPUT_DIR / "rfm_customers.csv")

    # Cohort: first purchase month vs retention
    d = data[[CUST_COL, DATE_COL, ID_COL]].dropna().copy()
    d["OrderMonth"] = d[DATE_COL].values.astype("datetime64[M]")
    first = d.groupby(CUST_COL)["OrderMonth"].min().rename("Cohort")
    d = d.join(first, on=CUST_COL)
    d["CohortIndex"] = ((d["OrderMonth"].dt.year - d["Cohort"].dt.year)*12 +
                        (d["OrderMonth"].dt.month - d["Cohort"].dt.month) + 1)
    cohort = (d.groupby(["Cohort","CohortIndex"])[CUST_COL]
                .nunique().unstack(fill_value=0))
    # Convert to retention %
    cohort_pct = cohort.divide(cohort.iloc[:,0], axis=0).round(3)
    cohort_pct.to_csv(OUTPUT_DIR / "cohort_retention.csv")


In [ ]:
# 9) ABC (Pareto) Analysis by Sales & Profit
# -------------------------
key = "Product Name" if "Product Name" in data.columns else "Product ID" if "Product ID" in data.columns else None
if key:
    prod = data.groupby(key, as_index=False)[["Sales","Profit"]].sum().sort_values("Sales", ascending=False)
    prod["Sales_Cum%"] = prod["Sales"].cumsum() / prod["Sales"].sum()
    prod["ABC_Sales"] = pd.cut(prod["Sales_Cum%"], bins=[0,0.8,0.95,1.0], labels=["A","B","C"], include_lowest=True)
    prod["Profit_Cum%"] = prod["Profit"].cumsum() / prod["Profit"].sum() if prod["Profit"].sum()!=0 else np.nan
    prod["ABC_Profit"] = pd.cut(prod["Profit_Cum%"], bins=[0,0.8,0.95,1.0], labels=["A","B","C"], include_lowest=True)
    prod.to_csv(OUTPUT_DIR / "abc_products.csv", index=False)

In [ ]:
# 10) Visuals (saved as PNG)
# -------------------------
def bar_save(series, title, fname, xlabel="", ylabel=""):
    plt.figure(figsize=(10,5))
    series.plot(kind="bar")
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.tight_layout()
    plt.savefig(OUTPUT_DIR / fname)
    plt.close()

# Sales by Month
sales_by_month = data.groupby(["Year","Month"], as_index=False)["Sales"].sum()
sales_by_month["YearMonth"] = pd.to_datetime(sales_by_month["Year"].astype(str) + "-" + sales_by_month["Month"].astype(str) + "-01")
sales_by_month = sales_by_month.set_index("YearMonth")["Sales"].sort_index()
plt.figure(figsize=(10,5))
sales_by_month.plot()
plt.title("Sales by Month")
plt.xlabel("Month")
plt.ylabel("Sales")
plt.tight_layout()
plt.savefig(OUTPUT_DIR / "sales_by_month.png")
plt.close()

# Category performance
if "Category" in data.columns:
    cat = data.groupby("Category")[["Sales","Profit"]].sum().sort_values("Sales", ascending=False)
    bar_save(cat["Sales"], "Sales by Category", "sales_by_category.png", ylabel="Sales")
    bar_save(cat["Profit"], "Profit by Category", "profit_by_category.png", ylabel="Profit")

# Region performance
if "Region" in data.columns:
    reg = data.groupby("Region")[["Sales","Profit"]].sum().sort_values("Sales", ascending=False)
    bar_save(reg["Sales"], "Sales by Region", "sales_by_region.png", ylabel="Sales")
    bar_save(reg["Profit"], "Profit by Region", "profit_by_region.png", ylabel="Profit")

# Weekday pattern
wk = data.groupby("Weekday")["Sales"].sum().reindex(
    ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
)
bar_save(wk, "Sales by Weekday", "sales_by_weekday.png", ylabel="Sales")

print(f"Done. Files saved in: {OUTPUT_DIR.resolve()}")


Done. Files saved in: /content/outputs
